In [1]:
import transformers
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from transformers import BartForConditionalGeneration
from transformers import BartTokenizer

from gradio import Interface
import pickle

2024-04-25 11:34:06.465700: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Context Summarization

In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")   # BART model 
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

406M params

In [3]:
summarizer = transformers.pipeline("summarization",
                      tokenizer=tokenizer,
                      model=model)

In [4]:
tokenizer.save_pretrained("my_tokenizer")
model.save_pretrained("my_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [5]:
ARTICLE = """
Name: Aisha Khan
Date: 16th April 2024
Gender: Female
Age: 62
Weight: 68 kg
CHIEF COMPLAINT: Left knee pain
HISTORY OF PRESENT ILLNESS: This is a 62-year-old South
Asian female with a two-month history of left knee pain, particularly
worse with stairs and walking long distances. Denies any recent
injury, redness, or swelling.
PAST MEDICAL HISTORY: Osteoporosis diagnosed 10 years ago.
Type 2 Diabetes controlled with diet and metformin.
PAST SURGICAL HISTORY: Cholecystectomy at age 50.
REVIEW OF SYSTEMS: No other significant findings.
CURRENT MEDICATIONS: Metformin 500mg twice daily for
diabetes. Calcium and Vitamin D supplements for osteoporosis.
ALLERGIES: Penicillin allergy (rash)
PHYSICAL EXAMINATION: Weight 68 kg, height 165 cm.
VITAL SIGNS: Blood pressure 120/78 mmHg, heart rate 72 bpm,
temperature 36.8Â°C (98.2Â°F), respiratory rate 14 breaths per
minute.
LOWER EXTREMITIES: Left knee joint with mild tenderness on
palpation, no swelling or redness. Decreased range of motion
compared to right knee.
IMPRESSION:
1. Left knee osteoarthritis
Doctor's Advice: Recommend physical therapy for strengthening
and pain management. Consider X-ray of the left knee to assess
further. Discuss potential pain medication options with the patient.
Schedule a follow-up appointment in 4 weeks to monitor progress.
"""

In [6]:
def predict(text):
    result = summarizer(text, max_length=200, min_length=100, do_sample=False)[0]['summary_text']
    return result

In [7]:
interface = Interface(
  fn=predict,
  inputs="textbox",
  outputs="textbox",
  title="Summarizing the Input Document from the user ",
  description="If user prefers to have a summarized document, this model would do that"
)

interface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [7]:
result = summarizer(ARTICLE, max_length=130, min_length=50, do_sample=False)

In [9]:
result[0]['summary_text']

"John Doe's medical history goes beyond the initial diagnoses of hypertension and appendectomy. It's important to explore any past illnesses, surgeries, injuries, or hospitalizations. A more detailed description of John's abdominal pain can help narrow down potential causes. Understanding John's diet and exercise habits can reveal areas for improvement."

#### Patient-Doctor Summarization

In [2]:
pat_doc_tokenizer = AutoTokenizer.from_pretrained("Charankumarpc/test-dialogue-summarization")  
pat_doc_model = AutoModelForSeq2SeqLM.from_pretrained("Charankumarpc/test-dialogue-summarization")

In [3]:
pat_doc_model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [5]:
pat_doc_tokenizer.save_pretrained("Conversation_TOK")

('Conversation_TOK/tokenizer_config.json',
 'Conversation_TOK/special_tokens_map.json',
 'Conversation_TOK/vocab.json',
 'Conversation_TOK/merges.txt',
 'Conversation_TOK/added_tokens.json',
 'Conversation_TOK/tokenizer.json')

In [6]:
pat_doc_model.save_pretrained("Conversation_MODEL")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [4]:
Pat_Doc_Conversation = """

Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  
Patient: No. 
Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?
Patient: No.  
Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as?
Patient: I am seventy six years old and identify as a white female.


"""

In [11]:
def preprocess_conversation(conversation):
    inputs = pat_doc_tokenizer(conversation, return_tensors="pt") 
    return inputs 

In [12]:
def summarize(conversation):
    inputs = preprocess_conversation(conversation)
    output = pat_doc_model.generate(**inputs)
    summary = pat_doc_tokenizer.decode(output[0], skip_special_tokens=True)
    
    return summary 

In [13]:
def generate_summary(pat_doc_model):
    output = pat_doc_model.generate(**inputs)
    summary = pat_doc_tokenizer.decode(output[0], skip_special_tokens=True)
    
    return summary 


In [14]:
inputs = preprocess_conversation(Pat_Doc_Conversation)

In [15]:
inputs

{'input_ids': tensor([[    0, 50118, 50118, 41152,    35,   653,  3291,    47,   124,    88,
             5,  8474,   452,     6,  2649,   116,  1437, 50118, 18276,  4843,
            35,    38,   376,    11,    13,    10, 40453,     9,   127,  1925,
          1164,  6150,     4,  1437, 50118, 41152,    35,    85,  1326,   101,
         12521,  6706,  1432,    62,    19,    47,    94,    86,  2624,   110,
         30960,     6, 30311,   139, 16880, 37830,     6, 30311,  1517,   368,
         13310,     6, 15671, 37994, 36866,  1809,     6, 28349, 16448,   179,
         10100,     8, 12855, 15109,     4,  1437,  6319,    47,  5324,   143,
          1022,    50,   109,    47,    33,   143,  1379,  2624,   209,   743,
           116,  1437,  1437, 50118, 18276,  4843,    35,   440,     4,  1437,
         50118, 41152,    35,  6319,    47,    56,   143, 11696,    50,  1855,
          5622,     6, 21768,     6, 15744,     6, 27214,     6, 23600,     6,
          7050,  2400,     6,  7050,  

In [16]:
summary = generate_summary(pat_doc_model, inputs)

TypeError: generate_summary() takes 1 positional argument but 2 were given

In [17]:
summary

NameError: name 'summary' is not defined

In [18]:
interface = Interface(
  fn=summarize,
  inputs="textbox",
  outputs="textbox",
  title="Summarizing the Conversation Between the Doctor and the Patient",
  description="After patients select a doctor based on their interest, they can engage in conversation with the chosen doctor. If users prefer to have a summary of the entire conversation afterward, this model will handle the task.",   
)

interface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
